In [5]:
#export CUDA_PATH=/usr/local/cuda
import numpy as np
import csv
from tqdm import trange
import os
import pickle
import json
from datetime import datetime
import augmentation_tools_spike_times as aug
import pandas as pd

from ml_genn import Network, Population, Connection
from ml_genn.callbacks import Checkpoint, SpikeRecorder, VarRecorder, Callback, OptimiserParamSchedule
from ml_genn.compilers import EventPropCompiler
from ml_genn.connectivity import Dense
from ml_genn.initializers import Normal
from ml_genn.neurons import LeakyIntegrate, LeakyIntegrateFire, SpikeInput
from ml_genn.optimisers import Adam
from ml_genn.serialisers import Numpy
from ml_genn.synapses import Exponential
from time import perf_counter

from ml_genn.utils.data import (calc_latest_spike_time, calc_max_spikes,
                                preprocess_tonic_spikes)

import nvsmi
import json
import opendatasets as od

from ml_genn.compilers.event_prop_compiler import default_params

from rawHD_dataset_loader_padded_spikes import rawHD_Loader

In [2]:
# Kaggle dataset directory
dataset = 'https://www.kaggle.com/datasets/thomasshoesmith/spiking-google-speech-commands/data'

# Using opendatasets to download SGSC dataset
od.download(dataset)

with open("rawHD_params.json", "r") as f: 
    params = json.load(f)
    
params["num_samples"] = None
    
x_train, y_train, x_test, y_test, x_validation, y_validation = rawHD_Loader(dir = os.getcwd() + params["dataset_directory"],
                                                                           num_samples=params["num_samples"],
                                                                           shuffle = True,
                                                                           shuffle_seed = 0,
                                                                           process_padded_spikes = False)


Skipping, found downloaded files in "./spiking-google-speech-commands" (use force=True to force download)


In [8]:
training_details = pd.read_csv(os.getcwd() + params.get("dataset_directory") + "training_details.csv")

speakers_list = np.array(list(training_details.loc[:, "Speaker"]))

In [22]:
y_train_speaker = speakers_list[:len(y_train)]

In [23]:
y_validation_speaker = speakers_list[len(y_train):]

In [25]:
len(y_validation_speaker)

1631

In [27]:
np.unique(speakers_list)

array([ 0,  1,  2,  3,  6,  7,  8,  9, 10, 11])

In [30]:
len(list(range(12)))

12